In [1]:
import torch

data_path = "../data/train_test_split/"

X_train_tensor = torch.load(data_path + "X_train.pt")
y_train_tensor = torch.load(data_path + "y_train.pt")
X_test_tensor = torch.load(data_path + "X_test.pt")
y_test_tensor = torch.load(data_path + "y_test.pt")

print(X_train_tensor[:5])
print(y_train_tensor[:5])
print(X_test_tensor[:5])
print(y_test_tensor[:5])

C:\Users\gurgel\AppData\Local\Temp\ipykernel_18220\499658517.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train_tensor = torch.load(data_path + "X_train.pt")
C:\Use

tensor([[44.0000,  0.3864,  0.2998,  ...,  2.1696,  2.7545, -0.0761],
        [26.0000, -0.6976, -0.6823,  ..., -0.5210, -0.4049, -0.7114],
        [52.0000, -0.6339, -0.6493,  ..., -0.5595, -0.6063,  0.1994],
        [ 8.0000, -0.8817, -0.9247,  ..., -1.0052, -0.3735, -0.6908],
        [49.0000,  1.1035,  1.0956,  ..., -0.3706, -0.4393,  0.4880]])
tensor([17, 12,  8,  7, 13])
tensor([[46.0000, -0.2138, -0.2730,  ..., -0.9899, -0.8668, -0.8599],
        [10.0000, -1.2046, -1.1377,  ..., -0.7581,  0.9737, -0.1045],
        [33.0000, -1.7444, -1.7738,  ..., -0.7142, -0.5751, -0.1734],
        [56.0000, -1.4034, -1.4371,  ..., -0.5832, -0.7459, -0.7243],
        [32.0000, -1.1190, -1.1041,  ..., -0.2222, -0.5311, -0.3397]])
tensor([13, 10,  0,  1,  3])


In [2]:
from torch.utils.data import DataLoader, TensorDataset

inputs = X_train_tensor.shape[1]
batch_size = 512
num_workers = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

print(train_loader.dataset.tensors)
print(test_loader.dataset.tensors)

(tensor([[ 4.4000e+01,  3.8642e-01,  2.9976e-01,  ...,  2.1696e+00,
          2.7545e+00, -7.6099e-02],
        [ 2.6000e+01, -6.9760e-01, -6.8235e-01,  ..., -5.2101e-01,
         -4.0487e-01, -7.1142e-01],
        [ 5.2000e+01, -6.3395e-01, -6.4926e-01,  ..., -5.5952e-01,
         -6.0628e-01,  1.9943e-01],
        ...,
        [ 4.0000e+00,  1.2202e+00,  1.2272e+00,  ...,  3.4117e-01,
         -4.1676e-02,  7.6974e-01],
        [ 1.9000e+01, -1.7210e+00, -1.6329e+00,  ..., -7.0155e-01,
          2.0413e-01, -1.5632e-01],
        [ 4.0000e+00, -3.7073e-01, -3.5631e-01,  ..., -7.3038e-01,
         -7.8024e-01, -4.3450e-01]]), tensor([17, 12,  8,  ...,  2,  5, 12]))
(tensor([[46.0000, -0.2138, -0.2730,  ..., -0.9899, -0.8668, -0.8599],
        [10.0000, -1.2046, -1.1377,  ..., -0.7581,  0.9737, -0.1045],
        [33.0000, -1.7444, -1.7738,  ..., -0.7142, -0.5751, -0.1734],
        ...,
        [35.0000,  1.3852,  1.5208,  ..., -0.2222, -0.7631, -0.7330],
        [56.0000, -0.8822, -0.91

In [3]:
from mlp import CsiNeuralNet

checkpoint_model = "mlp/main_model_epoch_46.pth"
save_path = "mlp/main_model.pth"
num_epochs = 1000
net = CsiNeuralNet(inputs, device, state_file=None)

result_model = net.train_model(train_loader, test_loader, save_path, num_epochs)

Training on device: cuda
Epoch 1/1000, Accuracy: 39.60%, Loss: 1.97753076, In 30.17s
Model saved for epoch 10 with accuracy 76.90%
Epoch 10/1000, Accuracy: 77.70%, Loss: 0.71874223, In 290.52s
Model saved for epoch 18 with accuracy 80.10%
Epoch 20/1000, Accuracy: 82.78%, Loss: 0.54054722, In 334.98s
Model saved for epoch 29 with accuracy 81.67%
Epoch 30/1000, Accuracy: 85.54%, Loss: 0.44496616, In 339.90s
Model saved for epoch 38 with accuracy 81.86%
Epoch 40/1000, Accuracy: 87.68%, Loss: 0.37363377, In 342.53s
Epoch 50/1000, Accuracy: 90.30%, Loss: 0.29223594, In 340.97s
Epoch 60/1000, Accuracy: 92.35%, Loss: 0.22995466, In 354.72s
Early stopping at epoch 69 due to accuracy
Training complete: Epochs: 69, Accuracy: 93.46%, Loss: 0.19752097811843755, In 2345.38s
